In [2]:
from nltk import pos_tag
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk 
from itertools import chain
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

ModuleNotFoundError: No module named 'pyLDAvis.gensim'

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r'C:/Users/Jordan/Desktop/Uni work/Advanced Machine Learning/clean_text.csv', encoding = 'utf-8-sig')

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

import spacy

# Pre-Processing

In [4]:
text = df['text']
text = list(text)


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

text_words = list(sent_to_words(text))

print(text_words[:1])

[['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [6]:
bigram = gensim.models.Phrases(text_words, min_count=1, threshold=10) 
trigram = gensim.models.Phrases(bigram[text_words], threshold=10)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[text_words[0]]])

['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']


In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Form Bigrams
text_words_bigrams = make_bigrams(text_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
text_lemmatized = lemmatization(text_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(text_lemmatized[:1])

[['folk', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [9]:
id2word = corpora.Dictionary(text_lemmatized)

# Create Corpus
texts = text_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [10]:
id2word[0]

'cytokine'

In [11]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('cytokine', 1),
  ('folk', 1),
  ('paste', 1),
  ('pfizerbiontech', 1),
  ('storm', 1)]]

# Basic LDA model

In [23]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=4, 
                     random_state=42,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='symmetric',
                     per_word_topics=True)

In [24]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"day" + 0.025*"news" + 0.017*"good" + 0.013*"need" + 0.011*"efficacy" '
  '+ 0.011*"trudeau_canada" + 0.008*"centre" + 0.007*"expert" + 0.007*"many" + '
  '0.007*"hour"'),
 (1,
  '0.094*"covaxin" + 0.047*"russia" + 0.034*"approval" + 0.030*"third" + '
  '0.025*"amp" + 0.022*"government" + 0.017*"shoot" + 0.013*"govt" + '
  '0.013*"indian" + 0.012*"emergency_authorisation"'),
 (2,
  '0.165*"sputnikv" + 0.113*"vaccine" + 0.042*"covid" + 0.028*"russian" + '
  '0.026*"sputnik" + 0.024*"case" + 0.019*"emergency" + 0.018*"moscow_russia" '
  '+ 0.016*"ontario_reporting" + 0.013*"covishield"'),
 (3,
  '0.112*"vaccine" + 0.110*"india" + 0.053*"covid" + 0.050*"moderna" + '
  '0.035*"dose" + 0.028*"covidvaccine" + 0.015*"pfizer" + 0.012*"available" + '
  '0.010*"ontario_reporte" + 0.010*"canada_stay"')]


In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.737425831423055

Coherence Score:  0.46371958015643766


In [26]:
pprint(lda_model.top_topics(corpus,topn=10))

[([(0.09403898, 'covaxin'),
   (0.04660271, 'russia'),
   (0.034192897, 'approval'),
   (0.030021282, 'third'),
   (0.025349906, 'amp'),
   (0.021796865, 'government'),
   (0.016594337, 'shoot'),
   (0.013373922, 'govt'),
   (0.012939873, 'indian'),
   (0.011738341, 'emergency_authorisation')],
  -6.333514872303752),
 ([(0.16524655, 'sputnikv'),
   (0.113237165, 'vaccine'),
   (0.04247302, 'covid'),
   (0.027627703, 'russian'),
   (0.02643301, 'sputnik'),
   (0.023882063, 'case'),
   (0.018542623, 'emergency'),
   (0.017754152, 'moscow_russia'),
   (0.01564999, 'ontario_reporting'),
   (0.013459804, 'covishield')],
  -9.680567749230942),
 ([(0.024663318, 'day'),
   (0.024621831, 'news'),
   (0.017119454, 'good'),
   (0.013144433, 'need'),
   (0.011248876, 'efficacy'),
   (0.010913699, 'trudeau_canada'),
   (0.007666552, 'centre'),
   (0.0074412036, 'expert'),
   (0.007417414, 'many'),
   (0.0070925136, 'hour')],
  -9.97349802509455),
 ([(0.1116414, 'vaccine'),
   (0.1104009, 'india'),


# Visualize the results

In [16]:
pyLDAvis.enable_notebook()
lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(lda_visualization)

# Trying to find ideal number of topics

In [27]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ---------- 
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model= LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, update_every=1, chunksize=100, passes=10, alpha='symmetric', per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=text_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=text_lemmatized, start=2, limit=23, step=2)
# Show graph
import matplotlib.pyplot as plt
limit=22; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()